In [ ]:
!echo $VIRTUAL_ENV

In [ ]:
!pip list

In [ ]:
import yfinance as yf

In [10]:
seaholdings = yf.Ticker("SE")

In [19]:
seaholdings.info['currentPrice']

106.26

In [25]:
# With multiple ticker symbols
watchlist = ["SE", "FRAGUAB.MX", "VAPU.L", "AGED.L"]
watchlist_info = yf.Tickers(watchlist)

In [20]:
watchlist_info.tickers["FRAGUAB.MX"].info['currentPrice']

525.0

In [22]:
watchlist_info.tickers["VAPU.L"].info

{'phone': '+353 1 6123226',
 'companyOfficers': [],
 'executiveTeam': [],
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 47.32,
 'open': 48.285,
 'dayLow': 46.76,
 'dayHigh': 48.285,
 'regularMarketPreviousClose': 47.32,
 'regularMarketOpen': 48.285,
 'regularMarketDayLow': 46.76,
 'regularMarketDayHigh': 48.285,
 'trailingPE': 19.736616,
 'volume': 17914,
 'regularMarketVolume': 17914,
 'averageVolume': 12329,
 'averageVolume10days': 27195,
 'averageDailyVolume10Day': 27195,
 'bid': 47.115,
 'ask': 47.205,
 'yield': 0.0,
 'totalAssets': 2013228544,
 'fiftyTwoWeekLow': 26.225,
 'fiftyTwoWeekHigh': 48.285,
 'allTimeHigh': 48.015,
 'allTimeLow': 16.915,
 'fiftyDayAverage': 42.1041,
 'twoHundredDayAverage': 37.571274,
 'navPrice': 47.5241,
 'currency': 'USD',
 'tradeable': False,
 'ytdReturn': 40.90718,
 'beta3Year': 1.1,
 'fundFamily': 'Vanguard Group (Ireland) Limited',
 'fundInceptionDate': 1569283200,
 'legalType': 'Exchange Traded Fund',
 'threeYearAverageReturn': 0.1547036,
 '

In [26]:
watchlist_info.tickers["AGED.L"].info

{'companyOfficers': [],
 'executiveTeam': [],
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 9.8475,
 'open': 9.865,
 'dayLow': 9.77,
 'dayHigh': 9.865,
 'regularMarketPreviousClose': 9.8475,
 'regularMarketOpen': 9.865,
 'regularMarketDayLow': 9.77,
 'regularMarketDayHigh': 9.865,
 'trailingPE': 17.253588,
 'volume': 48850,
 'regularMarketVolume': 48850,
 'averageVolume': 61781,
 'averageVolume10days': 112794,
 'averageDailyVolume10Day': 112794,
 'bid': 9.6225,
 'ask': 10.0,
 'bidSize': 0,
 'askSize': 0,
 'yield': 0.0,
 'totalAssets': 643296896,
 'fiftyTwoWeekLow': 6.7375,
 'fiftyTwoWeekHigh': 9.9225,
 'allTimeHigh': 9.9225,
 'allTimeLow': 3.866,
 'fiftyDayAverage': 9.61955,
 'twoHundredDayAverage': 8.906996,
 'navPrice': 9.8124,
 'currency': 'USD',
 'tradeable': False,
 'ytdReturn': 26.30066,
 'beta3Year': 0.82,
 'fundFamily': 'BlackRock Asset Management Ireland - ETF',
 'fundInceptionDate': 1473292800,
 'legalType': 'Exchange Traded Fund',
 'threeYearAverageReturn': 0.1229721,